In [1]:
import pickle, gzip
import pandas as pd
import os
import numpy as np

In [4]:
with gzip.open('input/stanfordData_test_p5.nlp', 'rb') as fout:
    while True:
        try:
            tmp = pickle.load(fout)
        except EOFError:
            print("EOF")
            break
print("Done!")

EOF
Done!


In [2]:
df_features = None
flip = True
base = 'input/collins_duffy/'
count = 0
for filename in os.listdir(base):
    if filename.endswith(".csv"): 
        tmpFrame = pd.read_csv(os.path.join(os.getcwd(), base, filename))
        print(os.path.join(os.getcwd(), base, filename))
        tmpFrame = tmpFrame.rename(columns={'cdNorm_st': 'cdNorm'+str(count)})
        if flip:
            df_features = tmpFrame
            flip = False
        else:
            df_features = df_features.merge(tmpFrame,how='inner',on='id')
        count+=1
        continue
    else:
        continue

/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/collins_duffy/collins_duffy_1e-1_sst.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/collins_duffy/collins_duffy_1e-1_st.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/collins_duffy/collins_duffy_1e-2_sst.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/collins_duffy/collins_duffy_1e-2_st.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/collins_duffy/collins_duffy_1e0_sst.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/collins_duffy/collins_duffy_1e0_st.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/collins_duffy/collins_duffy_2e-1_sst.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/collins_duffy/collins_duffy_2e-1_st.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/collins_duffy/collins_duffy_5e-1_sst.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/collins_duffy/collins_duffy_5e-1_st.csv
/Users/Matthew/Do

In [3]:
df_features.columns

Index(['cdNorm0', 'id', 'cdNorm1', 'cdNorm2', 'cdNorm3', 'cdNorm4', 'cdNorm5',
       'cdNorm6', 'cdNorm7', 'cdNorm8', 'cdNorm9', 'cdNorm10', 'cdNorm11',
       'cdNorm12', 'cdNorm13'],
      dtype='object')

In [4]:
def read_data(path_to_file):
    df = pd.read_csv(path_to_file)
    print ("Shape of base training File = ", df.shape)
    print("Shape of base training data after cleaning = ", df.shape)
    return df

df_train = read_data("input/train.csv")

Shape of base training File =  (404290, 6)
Shape of base training data after cleaning =  (404290, 6)


In [5]:
df_comp = df_train.merge(df_features,how='inner',on='id')
df_comp = df_comp.fillna(value=0)

In [6]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
from sklearn import preprocessing

In [7]:
le = preprocessing.LabelEncoder()
# >>> lb.fit_transform(['yes', 'no', 'no', 'yes'])

X = df_comp[['cdNorm0', 'cdNorm1', 'cdNorm2', 'cdNorm3', 'cdNorm4', 'cdNorm5',
       'cdNorm6', 'cdNorm7', 'cdNorm8', 'cdNorm9', 'cdNorm10', 'cdNorm11',
       'cdNorm12', 'cdNorm13']]

y = df_comp['is_duplicate']

y = le.fit_transform(y)

selector_1 = SelectKBest(chi2, k='all').fit(X,y)

In [8]:
selector_2 = SelectKBest(f_classif, k='all').fit(X,y)

In [9]:
selector_3 = SelectKBest(mutual_info_classif, k='all').fit(X,y)

In [10]:
print(selector_1.scores_)

[ 4805.72727281  6766.15137454  4832.98468866  6717.81066924   292.90579625
  5398.31570453  4714.81172705  6817.78841844  3622.87024336  6951.32883752
  4825.51842379  6739.54025863   927.52145     6803.12405878]


In [11]:
print(selector_2.scores_)

[ 34708.84764329  45689.75921777  36084.40829209  45259.30053561
    675.14014113  42516.36197518  32494.31485445  46108.85847876
  18149.89191184  47083.00872977  35530.66411392  45457.81962811
   2597.10118075  46750.70470034]


In [12]:
print(selector_3.scores_)

[ 0.10322086  0.12212759  0.11208143  0.12629059  0.08110915  0.08799524
  0.09915377  0.11911368  0.10322444  0.11436282  0.10319109  0.12461733
  0.09017436  0.11452788]


In [ ]:
'''
The two worst performs by a large margin are 5th from the beginning and 6th from the end, 
and 2nd from the end. As predicted these are all SST features for high values of lambda.

SST: 1e0
SST: 8e-1
SST: 5e-1
'''